In [1]:
import numpy as np 
import pandas as pd
import os 
import sys 
import torch 
import torchaudio 
import matplotlib.pyplot as plt 
#%matplotlib_inline

import IPython.display as ipd

import torchaudio_functions as Audio
from torchaudio_functions import *

# Dataset Exploritory Analysis 

The RAVDESS (Ryerson Audio-Visual Database of Emotional Speech and Song) is a widely used dataset for emotion classification using recorded speech because of its high quality and consistent audio quality. The dataset can be found at https://smartlaboratory.org/ravdess/ and more info can be found in this offical citation from the creators: 

Livingstone SR, Russo FA (2018) The Ryerson Audio-Visual Database of Emotional Speech and Song (RAVDESS): A dynamic, multimodal set of facial and vocal expressions in North American English. PLoS ONE 13(5): e0196391. https://doi.org/10.1371/journal.pone.0196391.

RAVDESS contains both audio and video but for this project i will only be using and discussions the audio-only portion of the dataset. The database contains audio from 24 actors (12 male, 12 female) each speaking 2 similar sentences in a neutral North American accent. Each statement is spoken in 8 different emotions/ expressions (calm, happy, sad, angry, fearful, suprise, and disgust). Each one is performed in 2 different levels of emotional intensity (normal, strong) and a neutral expression is added. All audio recordings have a sample rate of 48kHz with a bit depth of 16bit. There is a total of 1440 audio files (24 actors X 60 trials per actor).


RAVDESS does not come with any sort of metadata table with information on the recordings but instead the filename themselves have all the information. Each filename has a 7 part numerical identifier (ex. 03-01-04-01-01-02-12.wav). The identifiers represent the following: 

    1.) Modality (01 = full-AV, 02 = video-only, 03 = audio-only).
    2.) Vocal channel (01 = speech, 02 = song).
    3.) Emotion (01 = neutral, 02 = calm, 03 = happy, 04 = sad, 05 = angry, 06 = fearful, 07 = disgust, 08 = surprised).
    4.) Emotional intensity (01 = normal, 02 = strong). NOTE: There is no strong intensity for the ‘neutral’ emotion.
    5.) Statement (01 = “Kids are talking by the door”, 02 = “Dogs are sitting by the door”).
    6.) Repetition (01 = 1st repetition, 02 = 2nd repetition).
    7.) Actor (01 to 24. Odd numbered actors are male, even numbered actors are female).

So for example the file 03-01-04-01-01-02-12.wav contains the following metadata: 

    1.) Audio-only (03)
    2.) Speech (01)
    3.) Sad (04)
    4.) Normal Intensity (01)
    5.) "Kids are talking by the door" (01)
    6.) Second Repetition (02)
    7.) Actor-12 Male (12) 

# Creating Metadata table

To make the audio data easier to deal with I will create a Pandas data frame that will contain the file path of each audio file and linked to it's emotion as it will be are target variable. The audio files are each separated into their own folders by which actor performed them. 

In [5]:
# Create Dictionaries with Key Value pairs decoding file path 

modality = {'01':'full_av','02':'video_only','03':'audio_only'}
vocal_channel = {'01':'speech','02':'song'}
emotion = {'01':'neutral','02':'calm','03':'happy','04':'sad','05':'angry','06':'fearful','07':'disgust','08':'surprised'}
emotional_intensity = {'01':'normal','02':'strong'}
statement = {'01':'Kids are talking by the door','02':'Dogs are sitting by the door'}
reptition = {'01':'first_repitition','02':'second_repetition'}
def actor_f(num):
    if int(num)%2==0: return('female')
    else: return('male')

actors = sorted(os.listdir('/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24'))
actors.pop()
actors

['Actor_01',
 'Actor_02',
 'Actor_03',
 'Actor_04',
 'Actor_05',
 'Actor_06',
 'Actor_07',
 'Actor_08',
 'Actor_09',
 'Actor_10',
 'Actor_11',
 'Actor_12',
 'Actor_13',
 'Actor_14',
 'Actor_15',
 'Actor_16',
 'Actor_17',
 'Actor_18',
 'Actor_19',
 'Actor_20',
 'Actor_21',
 'Actor_22',
 'Actor_23']

In [8]:
audio_file_dict = {}
for actor in actors:
    actor_dir = os.path.join('/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24',actor)
    actor_files = os.listdir(actor_dir)
    actor_dict = [i.replace(".wav","").split("-") for i in actor_files]
    dict_entry = {os.path.join(actor_dir,i):j for i,j in zip(actor_files,actor_dict)}
    audio_file_dict.update(dict_entry)

audio_file_dict = pd.DataFrame(audio_file_dict).T
audio_file_dict.columns = ['modality','vocal_channel','emotion','emotional_intensity','statement','repetition','actor']


audio_file_dict.modality = audio_file_dict.modality.map(modality)
audio_file_dict.vocal_channel = audio_file_dict.vocal_channel.map(vocal_channel)
audio_file_dict.emotion = audio_file_dict.emotion.map(emotion)
audio_file_dict.emotional_intensity = audio_file_dict.emotional_intensity.map(emotional_intensity)
audio_file_dict.statement = audio_file_dict.statement.map(statement)
audio_file_dict.repetition = audio_file_dict.repetition.map(reptition)
audio_file_dict['actor_sex'] = audio_file_dict.actor.apply(actor_f)

audio_file_dict


,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,actor_sex
/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/Actor_01/03-01-08-02-02-01-01.wav,audio_only,speech,surprised,strong,Dogs are sitting by the door,first_repitition,01,male
/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/Actor_01/03-01-08-01-01-01-01.wav,audio_only,speech,surprised,normal,Kids are talking by the door,first_repitition,01,male
/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/Actor_01/03-01-05-01-02-01-01.wav,audio_only,speech,angry,normal,Dogs are sitting by the door,first_repitition,01,male
/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/Actor_01/03-01-06-01-02-02-01.wav,audio_only,speech,fearful,normal,Dogs are sitting by the door,second_repetition,01,male
/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/Actor_01/03-01-06-02-01-02-01.wav,audio_only,speech,fearful,strong,Kids are talking by the door,second_repetition,01,male
...,...,...,...,...,...,...,...,...
/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/Actor_23/03-01-03-02-02-02-23.wav,audio_only,speech,happy,strong,Dogs are sitting by the door,second_repetition,23,male
/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/Actor_23/03-01-03-01-01-02-23.wav,audio_only,speech,happy,normal,Kids are talking by the door,second_repetition,23,male
/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/Actor_23/03-01-02-02-01-01-23.wav,audio_only,speech,calm,strong,Kids are talking by the door,first_repitition,23,male
/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/Actor_23/03-01-02-01-02-01-23.wav,audio_only,speech,calm,normal,Dogs are sitting by the door,first_repitition,23,male


In [9]:
# Save dataframe for easy use in the future, export with Pandas.to_csv()

audio_file_dict.to_csv('/Users/stephen/Emotion_Dectection/data/RAVDESS/metadata.csv')

In [11]:
# Lets take a look at our target variables 
audio_file_dict['emotion'].value_counts()

surprised    184
angry        184
fearful      184
disgust      184
sad          184
happy        184
calm         184
neutral       92
Name: emotion, dtype: int64

The Dataset is all balanced except for the "neutral" emotion. This doesn't seem like it will be a problem so we'll leave it as is for now. 


# Turning the dataset into a Pytorch Dataset Object 
We can use Pytorch to turn our collection of audio files into a Pytorch object so it'll be easier to reuse. 
Since the intention is to input the data into a Neural Network, every observation but be the same length  

In [14]:
# make all audio files the same shape 
# make a list with all the audio files to easily see the shapes of each file
audio_files = []
for i in list(audio_file_dict.index):
    i, sr = torchaudio.load(i)
    audio_files.append(i)

# Iterate through the data and find the maximum and minimum length 
maxlen = 0
minlen = np.Inf
for i in audio_files:
    if i.shape[1]>maxlen:
        maxlen = i.shape[1]
    if i.shape[1]<minlen:
        minlen = i.shape[1]

minlen, maxlen

(140941, 253053)

Now we'll make a list of mel-spectrograms to get the sizes 

In [17]:
mel_spectrograms = []
for i in audio_files:
    mel_spect = torchaudio.transforms.MelSpectrogram(sample_rate=16000, n_fft=1024, hop_length=512, n_mels=64)(i)
    mel_spectrograms.append(mel_spect)


max_width, max_height = max([i.shape[2] for i in mel_spectrograms]), max([i.shape[1] for i in mel_spectrograms])

max_width, max_height

(495, 64)

In [ ]:
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, audio_file_dict):
        self.audio_file_dict = audio_file_dict.reset_index()

    def __len__(self):
        count = len(self.audio_file_dict)
        return count

    def __getitem__(self, index):
            path = self.audio_file_dict.reset_index()['index'][index]
            signal, sr = torchaudio.load(path)
            signal = torch.mean(signal, dim=0).unsqueeze(0)
            img = torchaudio.transforms.Spectrogram()(img)
            img = F.pad(img, [0, max_width - img.size(2), 0, max_height - img.size(1)])
            label = list(pd.get_dummies(self.audio_file_dict.reset_index().emotion).iloc[index].values)
            label = np.array(label)
            label = torch.from_numpy(label)
            return (img, label)

if __

    

# Visualizing the audio data
Here we'll use the Pytorch module torchaudio to convert the .wav files to tensor object so we can visualize the signal and then perform the necessary feature extractions to obtain data to train our model. 

In [ ]:
# declare a variable that will locate a wav file to extract from 
angry_file_row = Ravdess_df.loc[Ravdess_df['Emotions'] == 'angry']
test_file_row = angry_file_row.iloc[0]
test_file_row

In [ ]:
# based on Pytorch Documents 
# https://pytorch.org/tutorials/beginner/audio_feature_extractions_tutorial.html



# global variables 
n_fft = 2048
n_mels = 128
n_mfcc = 128
win_length = None 
hop_length = 512
sample_rate = None


# load audio file path to tenso 
waveform, sample_rate = torchaudio.load(test_file_row['Path'])

print_stats(waveform, sample_rate=sample_rate)
plot_waveform(waveform, sample_rate=sample_rate)
plot_specgram(waveform, sample_rate)
play_audio(waveform, sample_rate)

In [ ]:
spectrogram = T.Spectrogram(
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
)
# Perform transformation
spec = spectrogram(waveform)

print_stats(spec)
plot_spectrogram(spec[0], title='torchaudio')

In [ ]:
mel_spectrogram = T.MelSpectrogram(
    sample_rate=sample_rate,
    n_fft=n_fft,
    win_length=win_length,
    hop_length=hop_length,
    center=True,
    pad_mode="reflect",
    power=2.0,
    norm='slaney',
    onesided=True,
    n_mels=n_mels,
    mel_scale="htk",
)

melspec = mel_spectrogram(waveform)
plot_spectrogram(
    melspec[0], title="MelSpectrogram - torchaudio", ylabel='mel freq')

In [ ]:
mfcc_transform = torchaudio.transforms.MFCC(
    sample_rate=sample_rate,
    n_mfcc=n_mfcc,
    melkwargs={
      'n_fft': n_fft,
      'n_mels': n_mels,
      'hop_length': hop_length,
      'mel_scale': 'htk',
    })

mfcc = mfcc_transform(waveform)

plot_spectrogram(mfcc[0])

In [ ]:
pitch = torchaudio.functional.detect_pitch_frequency(waveform, sample_rate)
plot_pitch(waveform, sample_rate, pitch)
play_audio(waveform, sample_rate)

In [ ]:
pitch_feature = torchaudio.functional.compute_kaldi_pitch(waveform, sample_rate)
pitch, nfcc = pitch_feature[..., 0], pitch_feature[..., 1]

plot_kaldi_pitch(waveform, sample_rate, pitch, nfcc)
play_audio(waveform, sample_rate)

# Audio Feature Augmentation 
https://pytorch.org/tutorials/beginner/audio_feature_augmentation_tutorial.html

In [ ]:
import torchaudio.functional as F
import torchaudio.transforms as T

# creating a Pytorch dataframe odject


In [ ]:
from torch.utils.data import Dataset
# https://www.youtube.com/watch?v=88FFnqt5MNI&ab_channel=ValerioVelardo-TheSoundofAI

In [ ]:
# creating CSV with labels and path name 
# https://www.kaggle.com/shivamburnwal/speech-emotion-recognition


Ravdess = '/Users/stephen/Emotion_Dectection/data/RAVDESS/Audio_Speech_Actors_01-24/'

ravdess_directory_list = os.listdir(Ravdess)

file_emotion = []
file_path = []
for dir in ravdess_directory_list:
    # as their are 20 different actors in our previous directory we need to extract files for each actor.
    actor = os.listdir(Ravdess + dir)
    for file in actor:
        part = file.split('.')[0]
        part = part.split('-')
        # third part in each file represents the emotion associated to that file.
        file_emotion.append(int(part[2]))
        file_path.append(Ravdess + dir + '/' + file)
        
# dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])

# dataframe for path of files.
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)

# changing integers to actual emotions.
Ravdess_df.Emotions.replace({1:'neutral', 2:'calm', 3:'happy', 4:'sad', 5:'angry', 6:'fear', 7:'disgust', 8:'surprise'}, inplace=True)
Ravdess_df.head()

In [ ]:
Ravdess_df['Emotions'].value_counts()